In [35]:
import urllib

In [37]:
from requests import get
from bs4 import BeautifulSoup as Soup
import pandas as pd

num_pages = 500
genre = 'All'

search_pages = []

for i in range(num_pages):
    search_pages.append('https://www.metmuseum.org/art/collection/search#!?material=Handkerchiefs' \
                 '&offset=f{i*80}&perPage=80&pageSize=0&sortBy=Relevance&sortOrder=asc' \
                 '&searchField=All&showOnly=openAccess')
    
#     artwork__interaction artwork__interaction--download


def get_img(url: str):
    """get text descriptions from url"""
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8',
         'Connection': 'keep-alive'}

    d = get(url, headers=hdr)
    soup = Soup(d.content, 'html.parser')
#     print(soup)
    for foo in soup.findAll('div', {'class': 'result-card__container rich-text'}):
#         urllib.parse.unquote(
        print(type(foo.find('a', {'class':'result-card__link js-advanced-form'}).attrs['href'])) 
        
    descs = []
    for foo in soup.findAll('div', {'class': 'details'}):
        desc = foo.find('span', {'class': 'text'})
        if desc is None:
            # sometimes desc does not exist
            descs.append(None)
        else:
            # there exists 16 characters of white space and 'more' at the end of each description, we remove these
            descs.append(desc.text.strip()[:-16].replace('\r', ''))


for i in search_pages:
    get_img(i)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


KeyboardInterrupt: 

In [ ]:
num_pages = 20
start_page = 10
genre = 'All'

search_pages = []

for i in range(num_pages):
    search_pages.append(f'https://collections.mfa.org/search/objects/*/handkerchief/images?page={start_page+i}')

def get_img(url: str):
    """get text descriptions from url"""
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8',
         'Connection': 'keep-alive'}

    d = get(url, headers=hdr)
    soup = Soup(d.content, 'html.parser')
    domain = 'https://collections.mfa.org'
    for foo in soup.findAll('div', {'class': 'primaryMedia img-wrap'}):
        page = foo.find('a').attrs['href']
        url = f'{domain}{page}'
        item_id = page.split("/")[2]
        print(item_id)
        download = f'{domain}/download/{item_id}'
        d = get(download, headers=hdr)
        soup2 = Soup(d.content, 'html.parser')
        links = soup2.findAll('a', {'target': '_blank'})
        if links:
            dl_link = links[1].attrs['href']
            with open(f'assets/{item_id}.jpg', "wb") as f:
                f.write(get(dl_link).content)

for i in search_pages:
    get_img(i)

66164
66177
66180
66205
76751
79370
80562
90712
90819
90826
90860
90894
91384
93886
96146
98625
98630
98644
98659
98670
98678
98690
98702
100075
100085
100098
100107
100248
100287
100540
105895
105993
106603
106610


In [ ]:
import requests
from os.path  import basename

r = requests.get("xxx")
soup = BeautifulSoup(r.content)

for link in links:
    if "http" in link.get('src'):
        lnk = link.get('src')
        with open(basename(lnk), "wb") as f:
            f.write(requests.get(lnk).content)